In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

_数据格式_
```
+ 0 0 2 tcp 40 ------- 1 0.0 3.0 0 0
```
1. 动作: “+”：Enqueue；“-”：Dequeue；“r”：Receive；"d"：Drop
2. 时间
3. from_node
4. to_node
5. 封包的类型
6. 封包的大小（单位byte）
7. 标记标注
8. 属于哪个数据流
9. 来源端
10. 目的端
11. sequence number
12. ID

ref: https://blog.csdn.net/yhdyy123/article/details/74027683


In [26]:
file_path = "trace/newreno/5/newreno.tr"

def load_ack(path):
    with open(path, 'r') as f:
        file_lines = f.readlines()
    ack_seq=[]  #提取含字段 r 2 0的ack帧    
    for line in file_lines:
        line = line.strip().split()
        if line[4] != "ack" or line[2] != '2' or line[3] != '0':
            continue
        ack_seq.append([line[0], float(line[1]), int(line[10])])
        
    return ack_seq

def load_tcp(path):
    with open(path, 'r') as f:
        file_lines = f.readlines()
    tcp_seq=[]  #提取含字段 r 2 0的ack帧    
    for line in file_lines:
        line = line.strip().split()
        if line[4] != "tcp" or line[2] != '0' or line[3] != '2':
            continue
        tcp_seq.append([line[0], float(line[1]), int(line[10])])
        
    return tcp_seq

def load_cwd(path):
    with open(path, 'r') as f:
        file_lines = f.readlines()
    cwd_seq=[]  #提取含字段 r 2 0的ack帧    
    for i in range(len(file_lines)):
        line = file_lines[i]
        line = line.strip().split()
        if line[5] != "cwnd_":
            continue
        cwd_seq.append([float(line[0]), float(line[6])])
        
    return cwd_seq

ack_seq = load_ack(file_path)
tcp_seq = load_tcp(file_path)
cwd_seq = load_cwd(file_path)
cwd_seq[0:5]

[[0.0, 1.0], [0.1281, 2.0], [0.25792, 3.0], [0.2591, 4.0], [0.38774, 5.0]]

In [34]:
def get_tcp_enqueue(tcp_seq):
    tcp_enqueue_seq = []
    for item in tcp_seq:
        if item[0] != '+':
            continue
        tcp_enqueue_seq.append([item[1], item[2]])
    return np.array(tcp_enqueue_seq)

def get_ack_receive(ack_seq):
    ack_receive_seq = []
    for item in ack_seq:
        if item[0] != 'r':
            continue
        ack_receive_seq.append([item[1], item[2]])
    return np.array(ack_receive_seq)

tcp_enqueue_seq = get_tcp_enqueue(tcp_seq)
ack_receive_seq = get_ack_receive(ack_seq)
print(tcp_enqueue_seq[1], ack_receive_seq[0])

[0.128096 1.      ] [0.128096 0.      ]


In [5]:
def draw_plot(data1, label1, data2=None, label2=None, plot_title="Plot", save_fig=False):
    fig = plt.figure(figsize=(16,6))
    plt.plot(data1[:,0],data1[:,1],alpha=0.8,color='red',label=label1)
    if data2 is not None:
        plt.plot(data2[:,0],data2[:,1],alpha=0.8,color='blue',label=label2)
    plt.title(plot_title)
    plt.legend(loc='best')
    plt.xlabel("time")
    plt.ylabel("sequence number")
    plt.show()
    if save_fig:
        plt.savefig(plot_title+'.png')


In [63]:
def find_retr(cwd_seq, tcp_enqueue_seq, time_range=0.1):
    retr_seq = []
    pre = cwd_seq[0]
    idx = 0
    for i in range(len(cwd_seq)):
        if pre[1] > cwd_seq[i][1]: #降窗
            print(cwd_seq[i][1])
            tm = round(cwd_seq[i][0],3)
            tmp_seq = []
            while idx < len(tcp_enqueue_seq):
                if tcp_enqueue_seq[idx][0] > tm - time_range and tcp_enqueue_seq[idx][0] < tm + time_range:
                    tmp_seq.append([tcp_enqueue_seq[idx][0], tcp_enqueue_seq[idx][1]])
                idx += 1
            tmp_seq=np.array(tmp_seq)
            print(tmp_seq.shape)
            retr_seq.append(np.array(tmp_seq))
        pre = cwd_seq[i]
    return  np.array(retr_seq)

retr_seq = find_retr(cwd_seq, tcp_enqueue_seq)

309.0
(167, 2)
1.0
(0,)
155.001
(0,)
155.0
(0,)


E:\Anaconda\lib\site-packages\ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [59]:
print(retr_seq.shape)

(4,)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

<Figure size 1152x432 with 0 Axes>

In [60]:
for item in retr_seq:
    draw_plot(item, "tcp_enqueue_seq", plot_title="Retransmission")